## Crea recomendaciones de películas personalizadas en tiempo real con Amazon Personalize

Imagine que está construyendo un sitio web de recomendaciones. En su sitio web, los usuarios ven recomendaciones personalizadas de títulos de películas en tiempo real.
Como parte de su sitio, quiere generar recomendaciones de títulos de películas para los usuarios del sitio web. Estas recomendaciones de títulos de películas deben basarse en el historial de navegación y visualización de los usuarios.

En este laboratorio, aprenderá a utilizar **Amazon Personalize** para entrenar una solución de recomendaciones de títulos de películas. Utilizará el *SDK de AWS para Python* para preparar los datos; a continuación, creará una solución y una campaña, e implementará el modelo de recomendación en Amazon Personalize.

Para hacer recomendaciones, **Amazon Personalize** utiliza un modelo de aprendizaje automático que se entrena con sus datos. Los datos utilizados para entrenar el modelo se almacenan en conjuntos de datos relacionados en un grupo de conjuntos de datos. Cada modelo se entrena utilizando una receta que contiene un algoritmo para un caso de uso específico. En **Amazon Personalize**, un modelo entrenado se conoce como versión de la solución. Una versión de la solución se despliega para su uso en una campaña. Los usuarios de sus aplicaciones pueden recibir recomendaciones a través de la campaña. Por ejemplo, una campaña puede mostrar recomendaciones de películas en un sitio web o una aplicación donde el título mostrado se basa en los hábitos de visualización que formaban parte del conjunto de datos.

### Descargar y preparar el conjunto de datos

Los grupos de datos de Amazon Personalize son contenedores de datos. Un grupo de conjuntos de datos es una colección de conjuntos de datos relacionados (interacciones, usuarios y artículos). Existen tres tipos de conjuntos de datos en Amazon Personalize:

* *Interactions*: Este conjunto de datos almacena datos históricos y en tiempo real de las interacciones entre los usuarios y los artículos. Estos datos pueden incluir datos de impresiones y metadatos contextuales sobre el contexto de navegación de tus usuarios, como su ubicación o dispositivo (móvil, tableta, escritorio, etc.). Debe crear, como mínimo, un conjunto de datos de Interacciones.

* *Users*: Este conjunto de datos almacena metadatos sobre sus usuarios. Puede incluir información como la edad, el sexo o la afiliación, que puede ser importante en los sistemas de personalización.

* *Items*: Este conjunto de datos almacena metadatos sobre sus artículos. Puede incluir información como el precio, el tipo de SKU o la disponibilidad.
En este tutorial, sólo se utilizan los datos de Interacciones. Para el uso avanzado de otros tipos de conjuntos de datos, consulte [conjuntos de datos y esquemas](https://grouplens.org/datasets/movielens/).

Su modelo de Amazon Personalize se entrenará en el conjunto de datos MovieLens Latest Small que contiene 100.000 valoraciones y 3.600 aplicaciones de etiquetas aplicadas a 9.000 películas por 600 usuarios. El conjunto de datos de MovieLens está comisariado por GroupLens Research.



In [32]:
# data_dir = "data"
# !mkdir $data_dir

# !cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
# !cd $data_dir && unzip ml-latest-small.zip
# dataset_dir = data_dir + "/ml-latest-small/"
!ls $dataset_dir

links.csv  movies.csv  ratings.csv  README.txt	tags.csv


In [33]:
data_dir = "data"
dataset_dir = data_dir + "/ml-latest-small/"
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd

In [34]:
original_data = pd.read_csv(dataset_dir + '/ratings.csv')
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [35]:
original_data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


A partir de esto, se puede ver que hay un total de 100.836 entradas en el conjunto de datos, con 4 columnas, y cada celda almacenada como formato `int64`, con la excepción de rating que es un `float64`.

Recuerde que los datos que necesita son los de la interacción con el usuario, que en este caso son *userId*, *movieId* y *timestamp*. Este conjunto de datos tiene una columna adicional, *rating*, que puede ser eliminada del conjunto de datos después de haberla utilizado para centrarse en las interacciones positivas.

### Preparar los datos

En este paso, se definen dos variables en el conjunto de datos para filtrar las películas que no gustan y simular mejor los datos recogidos por una plataforma de vídeo bajo demanda (VOD).

Dado que se trata de un conjunto de datos de calificación de películas con retroalimentación explícita, incluye películas calificadas de 1 a 5. Para este tutorial, se desea incluir sólo los movimientos que "gustaron" a los usuarios, y simular un conjunto de datos implícito que sea similar a los datos recogidos por una plataforma de vídeo bajo demanda (VOD). Para ello, a continuación se filtran todas las interacciones por debajo de 2 sobre 5, y se crean dos variables EVENT_TYPE: click y watch. Todas las películas calificadas con 2 o más se asignan como clic, y todas las películas calificadas con 4 o más se asignan como clic y ver.

In [36]:
watched_df = original_data.copy()
watched_df = watched_df[watched_df['rating'] > 3]
watched_df = watched_df[['userId', 'movieId', 'timestamp']]
watched_df['EVENT_TYPE']='watch'

clicked_df = original_data.copy()
clicked_df = clicked_df[clicked_df['rating'] > 1]
clicked_df = clicked_df[['userId', 'movieId', 'timestamp']]
clicked_df['EVENT_TYPE']='click'

interactions_df = clicked_df.copy()
interactions_df = interactions_df.append(watched_df)
interactions_df.sort_values("timestamp", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')

In [37]:
interactions_df.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

### Crear el grupo de conjuntos de datos

Un grupo de conjuntos de datos es una colección de conjuntos de datos relacionados. Para este paso, se crea un nuevo grupo de conjuntos de datos denominado ***personalize-demo-movielens*** y se activa.


In [39]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-demo-movielens3"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-2:528038902135:dataset-group/personalize-demo-movielens3",
  "ResponseMetadata": {
    "RequestId": "a455f130-c541-491e-84a1-f1c1e5a4792e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Mar 2022 11:03:01 GMT",
      "x-amzn-requestid": "a455f130-c541-491e-84a1-f1c1e5a4792e",
      "content-length": "106",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Antes de poder utilizar el grupo de conjuntos de datos, éste debe estar activo. Ejecute el siguiente bloque de código y espere a que la salida imprima un estado ACTIVO.

**Nota**: El estado del grupo de datos se comprueba cada segundo, hasta un máximo de 3 horas.

In [40]:
%%time
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE
CPU times: user 10.8 ms, sys: 0 ns, total: 10.8 ms
Wall time: 1min


### Crear el esquema y el conjunto de datos

Amazon Personalize necesita un esquema para entender sus datos. El siguiente bloque de código crea el esquema apropiado para el conjunto de datos MovieLens y lo proporciona a Personalize. Este bloque de código también crea el conjunto de datos de interacciones dentro del grupo de conjuntos de datos. Personalize utiliza este conjunto de datos para entrenar el modelo de recomendación.

In [42]:
interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-demo-movielens-interactions_3",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-movielens-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-2:528038902135:schema/personalize-demo-movielens-interactions_3",
  "ResponseMetadata": {
    "RequestId": "fa39195f-982f-46e8-9242-d05a0b397b90",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Mar 2022 11:04:41 GMT",
      "x-amzn-requestid": "fa39195f-982f-46e8-9242-d05a0b397b90",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}
{
  "datasetArn": "arn:aws:personalize:us-east-2:528038902135:dataset/personalize-demo-movielens3/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "250ff8d8-485b-4fcf-b9c1-d91b4a384b50",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Mar 2022 11:04:42 GMT",
      "x-amzn-requestid": "250ff8d8-485b-4fcf-b9c1-d91b4a384b50",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttemp

### Configura tu bucket de Amazon S3 e importa los datos

In [45]:
# Este script crea un bucket de Amazon S3 con un 
# nombre [id de cuenta]-[región]-personalizedemoml.

session = boto3.session.Session()
region = session.region_name
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizedemoml"
print(bucket_name)
if region == "us-east-1":
    s3.create_bucket(Bucket=bucket_name)
else:
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
        )

528038902135-us-east-2-personalizedemoml


In [46]:
# A continuación, cargue los datos.
interactions_file_path = data_dir + "/" + interactions_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

#### Configurar la política del bucket de S3

En este paso, se configura la política del bucket de Amazon S3 para que Amazon Personalize pueda leer el contenido de su bucket de S3. Ejecuta el siguiente bloque de código para crear y adjuntar la política adecuada.

In [47]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'QJ5W9T98ZWY1NV7W',
  'HostId': 'lIgqM9EZnQRdut9L3gayIVPTaF2cE51UsV/q6ziEr5z5pGXwGeXkopYpQoxhw5tL2v+QGP6mhtk=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'lIgqM9EZnQRdut9L3gayIVPTaF2cE51UsV/q6ziEr5z5pGXwGeXkopYpQoxhw5tL2v+QGP6mhtk=',
   'x-amz-request-id': 'QJ5W9T98ZWY1NV7W',
   'date': 'Fri, 04 Mar 2022 11:06:30 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [49]:
iam = boto3.client("iam")

role_name = "PersonalizeRolePOC3"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::528038902135:role/PersonalizeRolePOC3


### Importar el conjunto de datos a Amazon Personalize

Recuerde que creó el grupo de conjuntos de datos y el conjunto de datos en el paso 2. Ahora, puede crear el trabajo de importación que carga los datos de Amazon S3 en Amazon Personalize para utilizarlos en su modelo.

In [50]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import1",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-2:528038902135:dataset-import-job/personalize-demo-import1",
  "ResponseMetadata": {
    "RequestId": "26202ced-9390-47be-8cf6-f6ee9c5abcdc",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Mar 2022 11:07:48 GMT",
      "x-amzn-requestid": "26202ced-9390-47be-8cf6-f6ee9c5abcdc",
      "content-length": "112",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [51]:
%%time
max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 20.8 ms, sys: 2.34 ms, total: 23.1 ms
Wall time: 3min


La salida informa del estado del trabajo cada minuto. Espere a que el estado de DatasetImportJob muestre ACTIVE en su cuaderno. Este paso tarda entre 10 y 15 minutos.

### Crear y evaluar una solución

En este módulo, se crea una solución de Amazon Personalize. Una solución se refiere a la combinación de una fórmula de Amazon Personalize, parámetros personalizados y una o más versiones de la solución (modelos entrenados). Una vez creada una solución con una versión de solución, puede crear una campaña para desplegar la versión de solución y obtener recomendaciones. Para crear una solución en Amazon Personalize, debes hacer lo siguiente:

* *Elegir una fórmula* - Una fórmula es un término de Amazon Personalize que especifica un algoritmo apropiado para entrenar para un caso de uso determinado. Para obtener más detalles, consulte [cómo elegir una fórmula](https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html).

* *Configurar una solución* - Personalice los parámetros de la solución y los hiperparámetros específicos de la receta para que el modelo satisfaga sus necesidades empresariales específicas. Para más detalles, vea Configurar una solución.

* *Crear una versión de la solución (entrenar un modelo)* - Entrenar el modelo de aprendizaje automático que Amazon Personalize utilizará para generar recomendaciones para sus clientes. Para obtener más detalles, consulte Creación de una versión de la solución.

* *Evaluar la versión de la solución* - Utilice las métricas que Amazon Personalize genera a partir de la nueva versión de la solución para evaluar el rendimiento del modelo. Para obtener más detalles, consulte Evaluación de la versión de la solución.

Amazon Personalize proporciona tres tipos de fórmulas, pero en este tutorial sólo se utilizará la fórmula *User_Personalization*.
 
La fórmula User-Personalization (aws-user-personalization) está optimizada para todos los escenarios de recomendación de *User_Personalization*. Al recomendar artículos, esta fórmula utiliza la exploración automática de artículos.
 
Con la exploración automática, Amazon Personalize prueba automáticamente diferentes recomendaciones de artículos, aprende de la forma en que los usuarios interactúan con estos artículos recomendados y potencia las recomendaciones de artículos que impulsan un mejor compromiso y conversión. Esto mejora el descubrimiento de artículos y el compromiso cuando los datos de interacción se actualizan con frecuencia y hay nuevos usuarios y nuevos artículos.
Se puede equilibrar cuánto explorar (donde los artículos con menos datos de interacciones o relevancia se recomiendan con más frecuencia) frente a cuánto explotar (donde las recomendaciones se basan en lo que conocemos o en la relevancia). Amazon Personalize ajusta automáticamente las recomendaciones futuras basándose en los comentarios implícitos de los usuarios.

In [52]:
# aws-user-personalization selected for demo purposes
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

En este paso, se crea la solución (modelo) y se configura para el entrenamiento del modelo. La configuración de la solución le permite personalizar los parámetros de la solución y los hiperparámetros específicos de la fórmula para que el modelo satisfaga sus necesidades empresariales específicas.

In [53]:
create_solution_response = personalize.create_solution(
    name = "personalize-demo-soln-user-personalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-2:528038902135:solution/personalize-demo-soln-user-personalization",
  "ResponseMetadata": {
    "RequestId": "825fee24-12e8-4ad7-b7eb-422360b6f7c0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Mar 2022 11:21:14 GMT",
      "x-amzn-requestid": "825fee24-12e8-4ad7-b7eb-422360b6f7c0",
      "content-length": "112",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Crear una versión de la solución (entrenar un modelo)

En el codigo anterior definiste una solución (modelo), ahora puedes entrenar el modelo. En Amazon Personalize, el entrenamiento del modelo se denomina creación de una versión de la solución. Puede pensar en cada versión como un modelo entrenado utilizando los conjuntos de datos actuales.

In [54]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-2:528038902135:solution/personalize-demo-soln-user-personalization/8d183d5f",
  "ResponseMetadata": {
    "RequestId": "67664a4d-b06b-4ef6-95c1-7afb9ae70f05",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Mar 2022 11:22:47 GMT",
      "x-amzn-requestid": "67664a4d-b06b-4ef6-95c1-7afb9ae70f05",
      "content-length": "128",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Para entrenar el modelo, ejecute el siguiente bloque de código y espere a que la versión de la solución imprima un estado ACTIVO. **Este paso tarda aproximadamente 40-50 minutos.**

In [55]:
%%time
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: ACTIVE
CPU times: user 133 ms, sys: 9.64 ms, total: 142 ms
Wall time: 21min 1s


### Evaluar la versión de la solución

Cuando crea una versión de la solución, Amazon Personalize genera métricas que puede utilizar para evaluar el rendimiento del modelo antes de crear una campaña y proporcionar recomendaciones. Las métricas le permiten ver los efectos de la modificación de los hiperparámetros de una solución. También puede utilizar las métricas para comparar los resultados entre soluciones que utilizan los mismos datos de entrenamiento pero que se crearon con diferentes fórmulas.

Amazon Personalize proporciona las siguientes métricas. Para cada métrica, los números más altos son mejores que los números más bajos.

* *coverage* - la proporción de artículos únicos recomendados de todas las consultas sobre el número total de artículos únicos en los conjuntos de datos de interacciones y artículos.


* *mean_reciprocal_rank_at_K* - la media de los rangos recíprocos de la primera recomendación relevante de las K principales recomendaciones sobre todas las consultas. Esta métrica es apropiada si está interesado en la única recomendación mejor clasificada.


* *normalized_discounted_cumulative_gain_at_K* - la ganancia descontada supone que las recomendaciones más bajas de una lista de recomendaciones son menos relevantes que las recomendaciones más altas. Por lo tanto, cada recomendación se descuenta (se le da un menor peso) por un factor que depende de su posición. Para obtener la ganancia acumulada descontada (DCG) en K, se suman todas las recomendaciones relevantes descontadas en las K primeras recomendaciones. La ganancia acumulativa descontada normalizada (NDCG) es la DCG dividida por la DCG ideal, de forma que la NDCG está entre 0 y 1. (La DCG ideal es aquella en la que las K primeras recomendaciones están ordenadas por relevancia). Amazon Personalize utiliza un factor de ponderación de 1/log(1 + posición), donde la parte superior de la lista es la posición 1. Esta métrica premia los artículos relevantes que aparecen cerca de la parte superior de la lista, porque la parte superior de una lista suele llamar más la atención.


* *precision_at_K* - El número de recomendaciones relevantes de las K primeras recomendaciones dividido por K. Esta métrica premia la recomendación precisa de los elementos relevantes.
Para obtener más información sobre estas métricas, consulte Evaluación de una versión de la solución


In [57]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-2:528038902135:solution/personalize-demo-soln-user-personalization/8d183d5f",
  "metrics": {
    "coverage": 0.0768,
    "mean_reciprocal_rank_at_25": 0.3324,
    "normalized_discounted_cumulative_gain_at_10": 0.3331,
    "normalized_discounted_cumulative_gain_at_25": 0.3629,
    "normalized_discounted_cumulative_gain_at_5": 0.319,
    "precision_at_10": 0.05,
    "precision_at_25": 0.0286,
    "precision_at_5": 0.0857
  },
  "ResponseMetadata": {
    "RequestId": "29756f46-4b06-4bd7-8cdb-854f801988fb",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Mar 2022 11:45:24 GMT",
      "x-amzn-requestid": "29756f46-4b06-4bd7-8cdb-854f801988fb",
      "content-length": "423",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


La ganancia acumulada descontada normalizada anterior indica que, con 5 artículos, hay un 31% de posibilidades de que una recomendación forme parte de la interacción de un usuario. Alrededor del 7% de los elementos recomendados son únicos, y hay una precisión de alrededor del 8,5% en los 5 elementos más recomendados.

Hay que tener en cuenta que este modelo utiliza datos de calificación para las interacciones porque MovieLens es un conjunto de datos explícito basado en calificaciones. Las marcas de tiempo también son del momento en que se calificó la película, no del momento en que se vio, por lo que el orden no es el mismo que el orden en que un espectador vería las películas. 

### Crear una campaña y obtener recomendaciones para los usuarios

Una campaña se utiliza para hacer recomendaciones a sus usuarios. En este módulo, usted crea una campaña desplegando su versión de la solución. Para desplegar una versión de la solución, se crea una campaña en la consola o llamando a la API CreateCampaign. Usted elige la versión de la solución que desea utilizar.

En los siguientes pasos, se crea la campaña y se espera a que el estado de la campaña se active. A continuación, utiliza su campaña para obtener recomendaciones de títulos de películas en tiempo real para los usuarios desde Amazon Personalize.

En este paso, se crea una campaña para la versión de la solución que se ha entrenado utilizando la receta de personalización de usuario. Esta fórmula incluye las opciones explorationWeight y explorationItemAgeCutOff para itemExplorationConfig. Para obtener más información, consulte [personalización del usuario](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html).

In [58]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-demo-camp",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.3",
	"explorationItemAgeCutOff": "30"
        }
    }
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

{
  "campaignArn": "arn:aws:personalize:us-east-2:528038902135:campaign/personalize-demo-camp",
  "ResponseMetadata": {
    "RequestId": "910c47d7-2a04-44e7-803e-466fe787cc19",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Mar 2022 11:47:14 GMT",
      "x-amzn-requestid": "910c47d7-2a04-44e7-803e-466fe787cc19",
      "content-length": "91",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}
Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


Espere a que el estado de la campaña se muestre como ACTIVO. Este paso puede tardar entre 10 y 15 minutos.

En el siguient codigo se utiliza una muestra, user_id aleatorio y se obtiene la recomendación para ese user_id. Tenga en cuenta que puede ver un resultado diferente porque esta recomendación es para un usuario aleatorio.

In [65]:
# Build a map to convert a movie id to the movie title
movies = pd.read_csv(dataset_dir + '/movies.csv', usecols=[0,1])
movies['movieId'] = movies['movieId'].astype(str)
movie_map = dict(movies.values)

# Getting a random user:
user_id, item_id = interactions_df[['USER_ID', 'ITEM_ID']].sample().values[0]

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = movie_map[item['itemId']]
    recommendation_list.append(title)
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['OriginalRecs'])
recommendations_df.head()

Recommendations for user:  380


,OriginalRecs
0,Avengers: Age of Ultron (2015)
1,The Hunger Games: Mockingjay - Part 2 (2015)
2,Minions (2015)
3,Elysium (2013)
4,Boyhood (2014)


En este ejemplo, puede ver las principales recomendaciones de títulos de películas para este usuario. Estas son las películas que el modelo cree que le gustarán al usuario basándose en los datos de interacción de los usuarios.

Genial. Has generado recomendaciones de títulos de películas en tiempo real para un usuario.

### Limpieza y próximos pasos

#### Eliminar los recursos de Amazon Personalize

In [ ]:
# Delete the campaign
personalize.delete_campaign(campaignArn=campaign_arn)
time.sleep(300)
print("delete_campaign done")
# Delete the solution
personalize.delete_solution(solutionArn=solution_arn)
time.sleep(60)
print("delete_solution done")
# Delete the interaction dataset
personalize.delete_dataset(datasetArn=interactions_dataset_arn)
time.sleep(60)
print("delete_dataset done")
# Delete the schema
personalize.delete_schema(schemaArn=interaction_schema_arn)
time.sleep(60)
print("delete_schema done")
# Delete the dataset group
personalize.delete_dataset_group(datasetGroupArn = dataset_group_arn)
time.sleep(60)
print("delete_dataset_group done")

#### Eliminar el bucket de Amazon S3 y las políticas de IAM

In [ ]:
# Empty S3 Bucket
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).delete()
# IAM policies should also be removed
iam = boto3.client("iam")
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/AmazonS3FullAccess", RoleName=role_name)
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess",RoleName=role_name)
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/service-role/IAMFullAccess",RoleName=role_name)
iam.delete_role(RoleName=role_name)

A continuación, detenga y elimine su instancia de Amazon SageMaker Notebook.

1. Abra la consola de SageMaker.
2. En Notebooks, seleccione instancias de Notebook.
3. Elija la instancia de notebook que creó para este tutorial y, a continuación, seleccione Acciones, Detener. La instancia de Notebook tarda varios minutos en detenerse. Cuando el estado cambie a Detenido, pase al siguiente paso.
4. Seleccione Acciones, y luego Eliminar.
5. Seleccione Eliminar.